# Project 13: Tokenization and Text Preprocessing

## Goal
Understand how text becomes data for transformers.

## Learning Objectives
- Character-level vs subword tokenization
- Byte-pair encoding (BPE)
- Special tokens (BOS, EOS, PAD, UNK)
- Creating data loaders

In [ ]:
# Setup
import numpy as np
from collections import Counter
import torch
from torch.utils.data import Dataset, DataLoader

print("Ready to explore tokenization!")